In [1]:
%matplotlib inline

# ロジスティック回帰

## 概要

### ロジスティック回帰とは

ロジスティック回帰分析は目的変数が1となる確率を予測します。

### ロジスティック回帰の活用例

* キャンペーンの反応率
* 土砂災害発生危険基準線の確率
* 医療における症例の発生確率

## ロジスティック回帰モデル(2クラス分類)

$$ \sigma\big( a \big)  = \frac{1}{1 + exp(-a)} $$

$$ p \big( C_{1}|\phi \big) = y\big( \phi \big) = \sigma\big( w^{T}\phi \big)  $$

$$ p \big( C_{1}|\phi \big) = \frac{1}{1 + exp(- w^{T}\phi)} $$

<div style="text-align: right;">p204. 4.3.2 ロジスティック回帰</div>

## 最尤法を用いたパラメータ(w)決定

### 交差エントロピー誤差関数(cross-entropy error function)

$$ \nabla E\big(w\big) = \sum_{n=1}^{N} \big( y_{n} - t_{n} \big) \phi_{n} $$

### ニュートン-ラフソン法による w の更新

$$ w^{(new)} = w^{(old)} - \big( \Phi^{T}R\Phi \big)^{-1}\Phi^{T}\big(y - t\big) $$
$$ = \big( \Phi^{T}R\Phi \big)^{-1} \big\{ \Phi^{T}R\Phi w^{(old)} - \Phi^{T}(y - t) \big\} $$
$$ = \big( \Phi^{T}R\Phi \big)^{-1}\Phi^{T}Rz $$

重み付き最小二乗問題に対する正規方程式の集合。

#### NxN の対角行列R

$$ R_{nn} = y_{n} \big( 1 - y_{n} \big) $$

パラメータ・ベクトル w に依存しているので、wが新しくなるたびに重み付け行列Rを計算し直す必要がある。

#### N次元ベクトルN

$$ z = \Phi w^{(old)} - R^{-1}\big( y - t \big)$$

このアルゴリズムを反復再重み付け最小二乗法( *iterative reweighted least squares method* = **IRLS** )

変数空間

$$ a = w^{T}\phi $$

内の線形化された問題の解としてIRLSを解釈することができる。
そのときzのn番目の要素に相当する

$$z_{n}$$

は、ロジスティックシグモイド関数を現在の点

$$w^{(old)}$$

の周りで局所線形近似して得られる空間での目的変数であると解釈できる。

$$ a_{n} \big(w\big)\simeq a_{n} \big( w^{(old)} \big) + \frac{da_{n}}{dy_{n}}|_{w^{(old)}}\big( t_{n} - y_{n} \big) $$
$$ = \phi_{n}^{T}w^{(old)} - \frac{(y_{n} - t_{n})}{y_{n}(1 - y_{n})} = z_{n} $$

## 参考書籍、ページ

* シュプリンガー パターン認識と機械学習
* [ロジスティック回帰分析](http://heartland.geocities.jp/ecodata222/ed/edj1-2-1-2.html)
* [確率密度比を用いた新しい機械学習アルゴリズム](https://www.google.com/url?q=http://www.ocw.titech.ac.jp/index.php%3Fmodule%3DGeneral%26action%3DDownLoad%26file%3D201027244-14-1-34.pdf%26type%3Dcal%26JWC%3D201027244&sa=U&ved=0ahUKEwihq_fl-LXOAhUJL8AKHT3FB4kQFggEMAA&client=internal-uds-cse&usg=AFQjCNGkrPBUHWw-gkJtPKE2EiTRjN7XFA)
* [TensorFlow-Examples](https://github.com/aymericdamien/TensorFlow-Examples)

## Tensor Board

```
python $PYTHON_PACKAGES/tensorflow/tensorboard/tensorboard.py --logdir=/tmp/mnist_logs
```

# コード

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## 準備

### パラメータの定義

In [3]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

### 手書きの数字画像を訓練データに使う

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
x = tf.placeholder(tf.float32, [None, 784], name="x-input") # mnist data image of shape 28*28=784
W = tf.Variable(tf.zeros([784, 10]), name="weights")
b = tf.Variable(tf.zeros([10]), name="bias")

## 活性化関数の定義

要はモデル

In [6]:
# 活性化関数
with tf.name_scope("Wx_b") as scope:
    y = tf.nn.softmax( tf.matmul(x, W) + b )

# matmulは行列積を計算する。↑は内積を計算。

In [7]:
y_ = tf.placeholder( tf.float32, [None,10], name="y-input" )  # 0〜9 10 classes

In [8]:
# for TensorBoard
w_hist = tf.histogram_summary("weights", W)
b_hist = tf.histogram_summary("biases", b)
y_hist = tf.histogram_summary("y", y)

## 損失関数の定義

交差エントロピー誤差関数のエラーを最小化する

In [9]:
# 損失関数
with tf.name_scope("xentropy") as scope:
    cross_entropy = -tf.reduce_sum( y_*tf.log(y) )
    ce_summ = tf.scalar_summary( "cross entropy", cross_entropy )

## オプティマイザの定義

In [10]:
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer( learning_rate ).minimize( cross_entropy )

In [11]:
init = tf.initialize_all_variables()

## モデルの訓練

In [16]:
# Launch the graph
with tf.Session() as sess:
    sess.run( init )

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int( mnist.train.num_examples / batch_size )

        for i in range( total_batch ):
            # feed = {x: mnist.test.images, y_: mnist.test.labels}
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # fit
            _, c = sess.run( [optimizer, cross_entropy], feed_dict={x: batch_xs, y_: batch_ys} )
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)

    print "Optimization Finished!"

    with tf.name_scope("test") as scope:
        # Test model
        correct_prediction = tf.equal( tf.argmax(y, 1), tf.argmax(y_, 1) )
        # accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        #accuracy = tf.reduce_mean(　tf.cast(correct_prediction, tf.float32)　)
        accuracy_summary = tf.scalar_summary("accuracy", accuracy)
        print "Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y_: mnist.test.labels[:3000]})

        # 全ての要約をマージしてそれらを /tmp/mnist_logs に書き出します。
        merged = tf.merge_all_summaries()
        writer = tf.train.SummaryWriter("/tmp/mnist_logs", sess.graph_def)

 Epoch: 0001 cost= 42.460622241
Epoch: 0002 cost= 31.676059714
Epoch: 0003 cost= 30.587358395
Epoch: 0004 cost= 29.562264427
Epoch: 0005 cost= 29.045376204
Epoch: 0006 cost= 28.798553118
Epoch: 0007 cost= 28.545274428
Epoch: 0008 cost= 28.201118325
Epoch: 0009 cost= 28.095019536
Epoch: 0010 cost= 27.824128557
Epoch: 0011 cost= 27.871425612
Epoch: 0012 cost= 27.589665217
Epoch: 0013 cost= 27.316248264
Epoch: 0014 cost= 27.571409637
Epoch: 0015 cost= 27.225286407
Epoch: 0016 cost= 27.038180164
Epoch: 0017 cost= 27.032229052
Epoch: 0018 cost= 27.101991416
Epoch: 0019 cost= 26.799124130
Epoch: 0020 cost= 26.858826623
Epoch: 0021 cost= 26.571702370
Epoch: 0022 cost= 26.657822528
Epoch: 0023 cost= 26.531379691
Epoch: 0024 cost= 26.544153503
Epoch:

 0025 cost= 26.540901189
Optimization Finished!
Accuracy: 0.896


## 分散TensorFlow

これらを分散環境で実行してみる


In [11]:
cluster = tf.train.ClusterSpec({"local": ["localhost:2222"]})
server = tf.train.Server(cluster, job_name="local", task_index=0)

In [12]:
with tf.device(tf.train.replica_device_setter(
    worker_device="/job:local/task:0",
    cluster=cluster)):
 
    global_step = tf.Variable(0)
 
    with tf.name_scope("dist_train") as scope:
        optimizer2 = tf.train.GradientDescentOptimizer( learning_rate ).minimize( cross_entropy, global_step=global_step )    
 
    saver = tf.train.Saver()
    summary_op = tf.merge_all_summaries()
    init_op = tf.initialize_all_variables()

In [13]:
# 訓練を監視するスーパーバイザを作成
sv = tf.train.Supervisor(is_chief=1,
                             logdir="/tmp/train_logs",
                             init_op=init_op,
                             summary_op=summary_op,
                             saver=saver,
                             global_step=global_step,
                             save_model_secs=600)
 
# スーパーバイザはセッション初期化をケアしてチェックポイントから復旧します。
sess = sv.prepare_or_wait_for_session(server.target)
 
# 入力パイプラインに対してキュー・ランナーを開始します（もしあれば）。
sv.start_queue_runners(sess)

# スーパーバイザがシャットダウンするまでループします（あるいは 1000000 ステップが完了するまで）。
step = 0
while not sv.should_stop() and step < 50000:
    # 訓練ステップを非同期に実行します。
    # *同期* 訓練をどのように遂行するかについての追加の詳細情報は `tf.train.SyncReplicasOptimizer` を参照。

    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    # fit
    _, step = sess.run([optimizer2, global_step], feed_dict={x: batch_xs, y_: batch_ys} )
